In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

In [6]:
import pandas as pd
import numpy as np
import os

In [4]:
# Some linux distributions of MKL and MP libraries will run operations multithreaded 
# under the hood. os.environ sets linux environmental variables that control this 
# behavior. Here we're limiting these libraries to using at most 2 threads (vs default ncpu)
# This way we can more predictably control system usage with python multiprocessing

# If you notice weird use of many mores than expected consider setting some of these environmental 
# variable. We can also consider setting them on a system wide level.
os.environ['MKL_NUM_THREADS'] = '1'
os.environ['GOTO_NUM_THREADS'] = '1'
os.environ['OMP_NUM_THREADS'] = '1'

This notebook assumes that you have a dataset with:
    1. Already deconvolved images
    2. Already found beads (will include a section about bead finding since it is an area for improvement)

In [14]:
# Load configuration for spot_calling
from seqfish_config import *

# Importing codebook (bunch of config options are setup in seqfish_config.py)
# WARNING - this file needs to vary in an experiment/microscopy specific fashion.
gids, cwords = zip(*cbook_dict.items())
bids, blanks = zip(*blank_dict.items())
genes = gids
cvectors = np.stack(cwords, axis=0)
cvectors.shape
nvectors = normalize(cvectors)

md_pth = '/scratch/hRedux_less_subset'
md = Metadata(md_pth)
posnames = md.posnames


In [101]:
# Import custom libraries
from metadata import *
from codestack_creation import *
from registration import *
from wrappers import *



In [22]:
from skimage.external.tifffile import TiffWriter
import subprocess
from shlex import split
from skimage import img_as_uint

def stkshow(images, fname='/home/rfor10/Downloads/tmp-stk.tif'):
    with TiffWriter(fname, bigtiff=False, imagej=True) as t:
        if len(images.shape)>2:
            for i in range(images.shape[2]):
                t.save(images[:,:,i].astype('uint16'))
        else:
            t.save(images.astype('uint16'))
#     java_cmd = ["java", "-Xmx5120m", "-jar", "/Users/robertf/ImageJ/ImageJ.app/Contents/Resources/Java/ij.jar"]
#     image_j_args = ["-ijpath", "/Users/robertf/ImageJ/", fname]
#     subprocess.Popen(java_cmd+image_j_args, shell=False,stdin=None,stdout=None,stderr=None,close_fds=True)

In [89]:

bead_pth = '/bigstore/Images2018/Robert/ca_celltypes/hybe_mcf10a_serumTitration_redux_deux_less_dense_cells_2018Jun18_2018Jun21/hdf3/'
tforms_xyy, tforms_zz, master_beadss = load_beads_find_tforms(bead_pth)

In [56]:
from collections import Counter

In [98]:
tforms_xy = defaultdict(list)
tforms_z = defaultdict(list)
tforms_z_raw = defaultdict(list)
# Iterate over all beads found in all hybes and fit gaussians then
# find the shift between the reference and nonreference
for k, hybe_dict in master_beads[p].items():
    #if str(k) in ref_substks:
        #sstk = ref_substks[str(k)]
        #ref_fit = gaussfitter.gaussfit(sstk[:,:,3])
    ref_fit = (k[0], k[1])
        #print(ref_fit)
    for h, dest_bead in hybe_dict.items():
            #if str(tuple(dest_bead)) in destination_substks[h]:
        try:
                #sstk = destination_substks[h][str(tuple(dest_bead))]
                #dest_fit = gaussfitter.gaussfit(sstk[:,:,3])
                #dest_fit = (dest_bead[0]+dest_fit[2]-3, dest_bead[1]+dest_fit[3]-3)
            dest_fit = (dest_bead[0], dest_bead[1])
            tforms_xy[h].append(np.subtract(ref_fit, dest_fit))
            tforms_z[h].append((k[2], np.subtract(k[2], dest_bead[2])))
            tforms_z_raw[h].append((k[2], dest_bead[2]))
        except:
            continue

In [105]:
h = 'hybe6'
dzdict = defaultdict(list)
for z, dz in tforms_z[h]:
    dzdict[z].append(dz)
dzdict = {k:(len(v), np.mean(v)+np.mean(tforms_z[h])) for k,v in dzdict.items()}
dzdict

{12: (6, 3.3855721393034828),
 14: (4, 4.052238805970149),
 16: (8, 4.677238805970149),
 17: (11, 4.915875169606513),
 13: (6, 3.5522388059701493),
 8: (4, 4.302238805970149),
 18: (6, 4.218905472636816),
 11: (4, 3.5522388059701493),
 10: (5, 3.1522388059701494),
 1: (1, 4.552238805970149),
 9: (6, 3.8855721393034823),
 5: (1, 4.552238805970149),
 6: (2, 4.552238805970149),
 7: (1, 5.552238805970149),
 2: (1, 3.5522388059701493),
 3: (1, 4.552238805970149)}

In [118]:
# Check that tforms and pseudo-max-projections result in well aligned images with bead stacks

# 
p = posnames[0]
#print(tforms_z)
bead_stk = pseudo_bead_stacks_byposition(p,md_pth, tforms_xyy, tforms_zz, zstart=14, k=1)

[13, 14, 15]
opening img_BL_site46_5_2_000000015_000000000_DeepBlue_000_015.tif[14, 15, 16]
opening img_BL_site46_5_2_000000016_000000000_DeepBlue_000_016.tif[13, 14, 15]
opening img_BL_site46_5_2_000000015_000000000_DeepBlue_000_015.tif[15, 16, 17]
opening img_BL_site46_5_2_000000017_000000000_DeepBlue_000_017.tif[13, 14, 15]
opening img_BL_site46_5_2_000000015_000000000_DeepBlue_000_015.tif[15, 16, 17]
opening img_BL_site46_5_2_000000017_000000000_DeepBlue_000_017.tif

In [ ]:
tforms_z

In [119]:
stkshow(bead_stk)

In [95]:
db = md.stkread(Position=p, Channel='DeepBlue', groupby='acq')

opening img_BR_site67_7_1_000000028_000000000_DeepBlue_000_028.tif

In [100]:
stkshow(db['hybe5_5'])

In [ ]:
cstk_save_dir = '/scratch/hRedux_less_subset/codestacks/'

if not os.path.exists(cstk_save_dir):
    os.makedirs(cstk_save_dir)

multi_z_wrapper(pos_subset[0])